In [1]:
### Import of modules

In [1]:
import numpy as np

from sklearn.model_selection import train_test_split

import gpflow
from gpflow.decors import params_as_tensors
from gpflow.mean_functions import MeanFunction
from gpflow.models import GPR

import tensorflow as tf

In [6]:
class ConvMeanFunction(MeanFunction):
    """
    Implement the mean function described in the paper
    """
    def __init__(self, input_dim, output_dim, seed=None):
        """
        :param input_dim: input dimension
        :param output_dim: output dimension
        :param seed: random seed
        """
        super().__init__()

        self.seed = seed
        self.rng = np.random.RandomState(self.seed)

        self.input_dim = input_dim
        self.output_dim = output_dim

    @params_as_tensors
    def __call__(self, X):
        """
        Feedforward pass

        :param X: NxD tensor
        :output Y: Nx1 tensor

        X[0] = "conv_kernel_height"
        X[1] = "conv_kernel_width"
        X[2] = "input_height"
        X[3] = "input_width"
        X[4] = "input_channel"
        X[5] = "output_height"
        X[6] = "output_width"
        X[7] = "output_channel"
        X[8] =  PC 
        X[9] =  PF
        X[10] = Logic clock
        X[11] = Memory access clock
        X[12] = DMA_eff
        X[13] = Memory access width
        X[14] = DW
        X[15] = First layer 
        X[16] = Last layer 
        """
        def my_func(X):
            Y = []
            N = len(X)
            for i in range(len(X)):
                T_weights = (X[i][0]*  X[i][1] * X[i][4] * X[i][5]* X[i][14])/ \
                            (X[i][9] * X[i][11] * x[i][13] * X[i][12])

                T_data = (X[i][2]*  X[i][3] * X[i][4] * X[i][14])/ \
                            (X[i][9] * X[i][11] * x[i][13] * X[i][12])

                T_load = T_weights + T_data

                T_compute = (X[i][7] * X[i][4] * X[i][2] * X[i][3] * X[i][0] * X[i][1])/ \
                            (X[i][8] * X[i][9] * X[i][10])

                T_store =  (X[i][5]*  X[i][6] * X[i][7] * X[i][14]) / \
                            (X[i][9] * X[i][11] * x[i][13] * X[i][12])
                time = 0.0
                if first:
                    time = T_load + T_compute
                elif last:
                    time = max(T_weights, T_compute) + T_store
                else:
                    time = max(T_weights, T_compute) 
                    
                Y.append(time)
            return np.array(Y).reshape((N,1))
            
        Y = tf.py_func(my_func, [X], tf.float64)
        
        return Y

In [7]:
X = [] # N x D where N is the number of samples and D is the length of the feature vector
Y = [] # N x 1 where N is the number of samples 

D = 1 # Dimensionality of the feature vector in this case 17

In [8]:
def build_model(X_t, Y_t):
    """
    param X_t: NxD tensor for training
    :param Y_t: Nx1 tensor for training

    :output a trained estimator
    """
    # Initialize the kernel k
    k =  gpflow.kernels.Matern52(D, lengthscales=0.3)
    # Initialize the mean m
    m = ConvMeanFunction(D, 1)

    # Create the model 
    estimator = gpflow.models.GPR(X_t, Y_t, k, mean_function=m)
    estimator.likelihood.variance = 0.01

    # Train the estimator
    gpflow.train.ScipyOptimizer().minimize(estimator)
    
    return estimator

In [0]:
# Compute the mean absolute error
abs_errors = []
for j in range(len(X)):
    # Select the data point that is currently used for testing
    X_test = X[j]
    X_test = np.array(X_test).reshape((1,len(X_test)))
    Y_test = Y[j]

    # Select the rest for training the estiamtor
    X_train = np.concatenate([X[0:j,:], X[j+1:len(X),:]])
    Y_train = np.concatenate([Y[0:j], Y[j+1:len(Y)]])
    Y_train = Y_train.reshape((len(Y_train),1))

    # Build the estimator
    estimator = build_model(X_train, Y_train)

    # Get the estimate and append it to the list
    estimate, _ = estimator.predict_y(X_test)
    abs_error=abs(estimate-Y_test)
    abs_errors.append(abs_error)

# Convert it to ms and measure the mean absolute error
mean_abs_error = np.mean(abs_errors)